In [1]:
!pip install datasets
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 19.4 MB/s 
     |████████████████████████████████| 212 kB 22.2 MB/s 
     |████████████████████████████████| 115 kB 20.8 MB/s 
     |████████████████████████████████| 182 kB 39.3 MB/s 
     |████████████████████████████████| 127 kB 27.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=a621e289006de2c02361a610cc17c06f4ddaab34e72d2772d52ae6e58dc1d2b2
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.6 MB/s 
     |████████████████████████████████| 7.6 MB 53.6 MB/s 


In [3]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=ffd7cd110b7067c99a2da846eb67bd05b53be9085d8aa7c19d00907c17e75c46
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [4]:
import datasets as ds

import pandas as pd
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoTokenizer,
    AutoModelWithLMHead,
    pipeline,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import nltk
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import json
import ast


"""### Log in to Hugging Face Hub"""

from huggingface_hub import notebook_login

In [5]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [6]:
"""### **T5 Fine-Tuning**
#### Load Dataset
"""

### Prepare data
# question_data = pd.read_excel('extra_data.xlsx')
question_data = pd.read_csv('relevant_sentences.csv')

metric = ds.load_metric("rouge")

"""#### Preprocess and create the dataset"""

def create_data_cols(data):
  for idx, row in data.iterrows():
    relevant_sentences = row["related_sentences"]
    
    data.at[idx, "context"] = relevant_sentences
    data.at[idx, "prompt"] = f'generateQuestion: {relevant_sentences}'

  return data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  if __name__ == '__main__':


In [7]:
question_data = create_data_cols(question_data)
question_data["question"] = question_data["Question"].map(str)

In [8]:
len(question_data)

959

In [10]:
# from https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

# x, x_test, y, y_test = train_test_split(xtrain,labels,test_size=0.2,train_size=0.8)
# x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75)

question_data = question_data[["prompt", "question"]]

train_data, val_data, test_data = train_validate_test_split(question_data)

qa_datasets: ds.DatasetDict = ds.DatasetDict({
    "train": ds.Dataset.from_pandas(train_data),
    "validation": ds.Dataset.from_pandas(val_data),
    "test": ds.Dataset.from_pandas(test_data),
})

In [20]:
print(qa_datasets)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'question', '__index_level_0__'],
        num_rows: 575
    })
    validation: Dataset({
        features: ['prompt', 'question', '__index_level_0__'],
        num_rows: 191
    })
    test: Dataset({
        features: ['prompt', 'question', '__index_level_0__'],
        num_rows: 193
    })
})


In [11]:
model_checkpoint = "t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 1024
max_target_length = 128

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [12]:
def preprocess_function(examples):
    inputs = examples["prompt"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["question"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenized_datasets = qa_datasets.map(preprocess_function, batched=True)

"""#### Load Model"""

# from transformers import (
#     AutoModelForQuestionAnswering,
#     T5ForConditionalGeneration,
#     DefaultDataCollator,
#     Seq2SeqTrainingArguments
# )

# model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, num_beams = 2, do_sample = True, max_length=max_input_length, repetition_penalty = 1.2)

"""#### Fine-Tuning Params"""

batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-qg-medium-hard-qns",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8, # 16 epochs causes some overfitting
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

nltk.download("punkt")

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # # Add mean generated length
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Cloning https://huggingface.co/mzhou08/t5-base-finetuned-qg-medium-hard-qns into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/850M [00:00<?, ?B/s]

Download file runs/Nov29_13-37-33_8878dbc8f1aa/events.out.tfevents.1669729063.8878dbc8f1aa.360.0: 100%|#######…

Download file runs/Nov29_13-37-33_8878dbc8f1aa/1669729063.3760636/events.out.tfevents.1669729063.8878dbc8f1aa.…

Download file training_args.bin: 100%|##########| 3.48k/3.48k [00:00<?, ?B/s]

...

Clean file pytorch_model.bin:   0%|          | 1.00k/850M [00:00<?, ?B/s]

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question, __index_level_0__, prompt. If question, __index_level_0__, prompt are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 575
  Num Epochs = 16
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2304
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,1.873365,29.874000,8.553600,26.243100,26.195900
2,No log,1.767489,31.437800,10.624400,27.484400,27.400500
3,No log,1.744908,35.477000,14.030500,31.401800,31.346400
4,1.961000,1.735383,33.678800,13.755200,30.518800,30.399200
5,1.961000,1.731612,34.491400,13.898900,30.755200,30.734100
6,1.961000,1.739665,34.725900,15.109000,31.065300,31.036200
7,1.480800,1.738052,33.789200,13.432700,30.063300,29.997600
8,1.480800,1.740367,35.111400,15.195700,30.566000,30.587400
9,1.480800,1.746199,36.046000,15.329000,31.872400,31.889300
10,1.480800,1.740012,34.614700,14.218100,31.124700,31.052600


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question, __index_level_0__, prompt. If question, __index_level_0__, prompt are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 191
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question, __index_level_0__, prompt. If question, __index_level_0__, prompt are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 191
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question, __index_level_0__, prompt. If question, __index_level_0__, prompt are not expecte

TrainOutput(global_step=2304, training_loss=1.4414921866522894, metrics={'train_runtime': 1229.0181, 'train_samples_per_second': 7.486, 'train_steps_per_second': 1.875, 'total_flos': 1973463632640000.0, 'train_loss': 1.4414921866522894, 'epoch': 16.0})

In [ ]:
trainer.push_to_hub()

In [16]:
def _construct_prompt(
        #keywords,
        relevant_sentences
    ) -> str:
        #keywords = ast.literal_eval(keywords)
        #keywords = " ".join(keywords)      

        return f'generateQuestion: context:{relevant_sentences}'


def generate_question(
        #keywords,
        relevant_sentences
    ) -> str:
        input = _construct_prompt(relevant_sentences)

        encoded_input = tokenizer([input],
                                    return_tensors='pt',
                                    max_length=1024,
                                    truncation=True)
        encoded_input = encoded_input.to("cuda:0")
        output = model.generate(input_ids = encoded_input.input_ids,
                                    attention_mask = encoded_input.attention_mask, num_beams = 2, do_sample = True, max_length=128, repetition_penalty = 1.2)
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        #output2 = tokenizer.decode(output[1], skip_special_tokens=True)
        #output3 = tokenizer.decode(output[2], skip_special_tokens=True)
        #output4 = tokenizer.decode(output[3], skip_special_tokens=True)
        #output5 = tokenizer.decode(output[4], skip_special_tokens=True)

        return output

In [17]:
generate_question("Dempsey was born in Nacogdoches, Texas, and, for much of his childhood, his family lived in a trailer park, where he and his siblings grew up playing soccer with Hispanic immigrants. In his teens, Dempsey maintained these ties playing in a local Mexican-dominated adult league. Dempsey is of Irish descent on his father's side. His older brother Ryan was offered a tryout for the Dallas Texans, an elite youth soccer club, and brought Clint, who was noticed and recruited while passing time juggling a ball on the sidelines. Dempsey became a standout on the team at an early age but had to quit due to his family's time and money constraints as his eldest sister Jennifer was becoming a ranked youth tennis player.")

"Did Dempsey's older brother bring Clint?"

In [18]:
def generate_question_1(
        prompt
    ) -> str:
        #input = _construct_prompt(keywords, relevant_sentences)

        encoded_input = tokenizer([prompt],
                                    return_tensors='pt',
                                    max_length=1024,
                                    truncation=True)
        encoded_input = encoded_input.to("cuda:0")
        output = model.generate(input_ids = encoded_input.input_ids,
                                    attention_mask = encoded_input.attention_mask, num_beams = 4, do_sample = True, max_length=128, num_return_sequences = 1, repetition_penalty = 1.2)
        #print(output)
        print(prompt)
        
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        print(output)
        print()

        return output

In [19]:
print(tokenized_datasets)
for prompt in tokenized_datasets["test"]['prompt']:
  generate_question_1(prompt)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'question', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 575
    })
    validation: Dataset({
        features: ['prompt', 'question', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 191
    })
    test: Dataset({
        features: ['prompt', 'question', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 193
    })
})
generateQuestion: There is an array of ancient sculptured temples, modern cities, scenic hill ranges, forests and beaches. Karnataka has been ranked as the fourth most popular destination for tourism among the states of India. Karnataka has the second highest number of nationally protected monuments in India, second only to Uttar Pradesh, in addition to 752 monuments protected by the State Directorate of Archaeology and Museums.
What is the highest number of nationally protected monuments in India?

generateQues